In [34]:
# Import libraries
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

from tqdm import tqdm
import time
import json

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
def getresponse(url):
    headers = {
        "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36",
        "Accept-Language":"en-US,en;q=0.9,is;q=0.8,da;q=0.7",
        "Referer":"https://www.google.com/",
        "DNT":"1",
    }
    res = requests.get(url, headers=headers)
    res.raise_for_status()
    return res

def percent2float(p):
    return float(p.strip('%'))/100  


In [3]:
# get field names
url = "https://etfdb.com/screener/"
response = getresponse(url)
soup = BeautifulSoup(response.text, 'lxml')

field_list = soup.find("ul", attrs={"class":"nav nav-pills"})
fields = [li.text.replace("New","").strip().replace(" ", "-").lower() for li in field_list.find_all("li")]
fields 

['overview',
 'returns',
 'fund-flows',
 'expenses',
 'esg',
 'dividends',
 'risk',
 'holdings',
 'taxes',
 'technicals',
 'analysis',
 'realtime-ratings']

In [4]:
# choose field names to fetch
fields = [
    'overview',
    'returns',
#     'fund-flows',
    'expenses',
    'esg',
    'dividends',
    'risk',
    'holdings',
#     'taxes',
#     'technicals',
#     'analysis',
    'realtime-ratings'
]

In [37]:
# fetch data
page_max = 120

# open browser
options = webdriver.ChromeOptions()
options.add_argument('--headless')
service = Service(ChromeDriverManager().install())


data = {}
for idx, field in enumerate(fields):
    
    print(f"\nFetcing data for field: {field} ({idx + 1} of {len(fields)})")
    
    field_data = pd.DataFrame()
    browser = webdriver.Chrome(service=service, options=options)
    
    for page in tqdm(range(page_max)):

        # update url with page and field
        url = f"https://etfdb.com/screener/#tab={field}&page={page + 1}"

        # open url and get page source
        browser.get(url)
        browser.refresh()
        time.sleep(3)
        
        html = browser.page_source

        # get page table as df from source
        soup = BeautifulSoup(html, features="html.parser")
        page_table = soup.find(id = "mobile_table_pills").find("div", attrs={"data-screener-tab": field})
        page_df = pd.read_html(str(page_table))[0].iloc[:-1,:]

        # store current page df with field data
        field_data = pd.concat([field_data, page_df], ignore_index=True)

        # quit browser
        

    data[field] = field_data.to_dict(orient="list")
    browser.quit()




Fetcing data for field: overview (1 of 8)


100%|█████████████████████████████████████████| 120/120 [08:24<00:00,  4.21s/it]



Fetcing data for field: returns (2 of 8)


100%|█████████████████████████████████████████| 120/120 [08:26<00:00,  4.22s/it]



Fetcing data for field: expenses (3 of 8)


...

100%|█████████████████████████████████████████| 120/120 [08:48<00:00,  4.41s/it]



Fetcing data for field: holdings (7 of 8)


100%|█████████████████████████████████████████| 120/120 [08:29<00:00,  4.25s/it]



Fetcing data for field: realtime-ratings (8 of 8)


100%|█████████████████████████████████████████| 120/120 [08:30<00:00,  4.25s/it]


In [39]:
# with open("Data/ETFdata_etfdb_raw.json", "w") as outfile:
#     # writing to json file
#     json.dump(data, outfile)

In [46]:
for field, item in data.items():
    print(f"{field}:\n n={len(item.get('Symbol'))}\t n_unique = {len(np.unique(item.get('Symbol')))}")

overview:
 n=2995	 n_unique = 2994
returns:
 n=2995	 n_unique = 2994
expenses:
 n=2995	 n_unique = 2994
esg:
 n=2995	 n_unique = 2994
dividends:
 n=2995	 n_unique = 2994
risk:
 n=2995	 n_unique = 2994
holdings:
 n=2995	 n_unique = 2994
realtime-ratings:
 n=2995	 n_unique = 2994
